In [2]:
import pandas as pd


dataframe = pd.read_csv("Exmo_BTCEUR_1h.csv", header=1)

dataframe.head()

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume EUR
0,1.600326e+09,2020-09-17 07-AM,BTCEUR,9246.59,9251.27,9242.75,9251.27,0.00670,61.95
1,1.600322e+09,2020-09-17 06-AM,BTCEUR,9302.08,9318.55,9242.95,9246.59,0.07678,711.82
2,1.600319e+09,2020-09-17 05-AM,BTCEUR,9281.57,9322.23,9274.79,9302.08,0.09128,848.16
3,1.600315e+09,2020-09-17 04-AM,BTCEUR,9273.26,9289.01,9259.86,9281.57,0.07866,729.78
4,1.600312e+09,2020-09-17 03-AM,BTCEUR,9307.35,9309.64,9273.26,9273.26,0.09925,922.51


In [8]:
dataframe["Open hour before"] = dataframe["Open"].shift(-1)
dataframe["Open_changes"] = (dataframe["Open"] / dataframe["Open hour before"]) - 1

dataframe = dataframe.dropna()
dataframe.head()

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume EUR,Open day before,Open changed percentage,Open hour before,Open_changes
0,1.600326e+09,2020-09-17 07-AM,BTCEUR,9246.59,9251.27,9242.75,9251.27,0.00670,61.95,9302.08,-0.005965,9302.08,-0.005965
1,1.600322e+09,2020-09-17 06-AM,BTCEUR,9302.08,9318.55,9242.95,9246.59,0.07678,711.82,9281.57,0.002210,9281.57,0.002210
2,1.600319e+09,2020-09-17 05-AM,BTCEUR,9281.57,9322.23,9274.79,9302.08,0.09128,848.16,9273.26,0.000896,9273.26,0.000896
3,1.600315e+09,2020-09-17 04-AM,BTCEUR,9273.26,9289.01,9259.86,9281.57,0.07866,729.78,9307.35,-0.003663,9307.35,-0.003663
4,1.600312e+09,2020-09-17 03-AM,BTCEUR,9307.35,9309.64,9273.26,9273.26,0.09925,922.51,9338.39,-0.003324,9338.39,-0.003324


In [14]:
changes = dataframe["Open_changes"]

In [18]:
X = []
Y = []

for i in range(0, len(changes)):
    Y.append(changes[i])
    X.append(changes[i+1:i+20000][::-1])
    break

0
